# **Writing and reading problem files**

***write_read.ipynb***

In this notebook, we create a basic problem, save it to a file and then read that file into a new problem and solve it.

&copy; Copyright 2025 Fair Isaac Corporation

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0.
 
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

This example uses FICO&reg; Xpress software. By running it, you agree to the Community License terms of the [Xpress Shrinkwrap License Agreement](https://www.fico.com/en/shrinkwrap-license-agreement-fico-xpress-optimization-suite-on-premises) with respect to the FICO&reg; Xpress software. See the [licensing options](https://www.fico.com/en/fico-xpress-trial-and-licensing-options) overview for additional details and information about obtaining a paid license.

In [ ]:
# Install the xpress package
%pip install -q xpress

Create a basic Xpress problem with five variables of various types, set an objective and add a set of three constraints:

$$
\min c_{1} + c_{2}
$$

subject to:

$$
c_{1}^2 + c_{2}^2 \leq 6 \\
2c_{1} + 3c_{2} + c_{3} = 2 \\
-c_{3}^2 + c_{4}^2 + c_{5}^2 \leq 0 \\
$$

with the following bounds:

$$
-\inf \leq c_{1} \leq \inf \\
-\inf \leq c_{2} \leq 200 \\
0 \leq c_{3} \leq \inf \\
0 \leq c_{4} \leq 6 \\
0 \leq c_{5} \leq \inf \\

$$

with $c_{3}$ being *partially integer* with a threshold of 10, $c_{4}$ a *semi-continuous* variable with a <tt>threshold</tt> of 3 and upper bound equal to 6, and $c_{5}$ being *integer*.

In [1]:
import xpress as xp

p = xp.problem()

c1 = p.addVariable(name="C1", lb=-xp.infinity, ub=xp.infinity)
c2 = p.addVariable(name="C2", lb=-xp.infinity, ub=200)
c3 = p.addVariable(name="C3", vartype=xp.partiallyinteger, threshold=10)    # Integer up to 10, then continuous
c4 = p.addVariable(name="C4", vartype=xp.semicontinuous, threshold=3, ub=6) # A 'hole' between 0 and 3, then continuous up to 6
c5 = p.addVariable(name="C5", vartype=xp.integer)

p.setObjective(c1 + c2)

p.addConstraint(c1**2 + c2**2 <= 6,
                2 * c1 + 3 * c2 + c3 == 2,
                -c3**2 + c4**2 + c5**2 <= 0)

After a problem is built by the user, the [problem.write()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.write.html) method generates and exports the problem as an LP file named "example0" into the active working directory. The method can be used to write in either LP or MPS format, where LP is more human readable and best fit for debugging and model validation, while the MPS format is recommended for numerical experiments and reproduceability.

This file can then be opened for debugging purposes or for a visual validation of the problem formulation.

In [2]:
p.write("example0", "lp")

A second problem is created and loaded by reading the generated LP file using the [problem.read()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.read.html) method, with the **filename with extension** being passed as an argument. Then, the problem is optimized, and the objective and solutions values of the optimal solution are printed.

In [ ]:
p2 = xp.problem()

p2.read("example0.lp")

p2.optimize()

print("Objective value:", p2.attributes.objval)
print("Solution:", p2.getSolution())